# Upload Vehicle dataset

In [1]:
import pandas as pd
import numpy as np

In [2]:
######################## vehicle dataset
df = pd.read_csv('/Users/fabianafazio/Documents/GitHub/BP24/Fabiana/Data/fraud_oracle.csv')
df.head()

,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,...,AgeOfVehicle,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange_Claim,NumberOfCars,Year,BasePolicy
0,Dec,5,Wednesday,Honda,Urban,Tuesday,Jan,1,Female,Single,...,3 years,26 to 30,No,No,External,none,1 year,3 to 4,1994,Liability
1,Jan,3,Wednesday,Honda,Urban,Monday,Jan,4,Male,Single,...,6 years,31 to 35,Yes,No,External,none,no change,1 vehicle,1994,Collision
2,Oct,5,Friday,Honda,Urban,Thursday,Nov,2,Male,Married,...,7 years,41 to 50,No,No,External,none,no change,1 vehicle,1994,Collision
3,Jun,2,Saturday,Toyota,Rural,Friday,Jul,1,Male,Married,...,more than 7,51 to 65,Yes,No,External,more than 5,no change,1 vehicle,1994,Liability
4,Jan,5,Monday,Honda,Urban,Tuesday,Feb,2,Female,Single,...,5 years,31 to 35,No,No,External,none,no change,1 vehicle,1994,Collision


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15420 entries, 0 to 15419
Data columns (total 33 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Month                 15420 non-null  object
 1   WeekOfMonth           15420 non-null  int64 
 2   DayOfWeek             15420 non-null  object
 3   Make                  15420 non-null  object
 4   AccidentArea          15420 non-null  object
 5   DayOfWeekClaimed      15420 non-null  object
 6   MonthClaimed          15420 non-null  object
 7   WeekOfMonthClaimed    15420 non-null  int64 
 8   Sex                   15420 non-null  object
 9   MaritalStatus         15420 non-null  object
 10  Age                   15420 non-null  int64 
 11  Fault                 15420 non-null  object
 12  PolicyType            15420 non-null  object
 13  VehicleCategory       15420 non-null  object
 14  VehiclePrice          15420 non-null  object
 15  FraudFound_P          15420 non-null

In [ ]:
# Loop through each column and print the unique classes
for column in df.columns:
    unique_classes = df[column].unique()
    num_classes = len(unique_classes)
    print(f"Column '{column}':")
    print(f"Number of unique classes: {num_classes}")
    print(f"Classes: {unique_classes}\n")



# Change y_train to the label column you want to explore (Is there Fraud?)

In [7]:
#y_train
y = df["FraudFound_P"]
y = y.astype('category')

# X_train
X = df.drop(columns=['FraudFound_P'])


# Use OneHotEncoder
- this changes categorical into binary columns

In [10]:
# USE OneHotEncoder to change categorical into binary columns
from sklearn.preprocessing import OneHotEncoder

# Separate categorical and numerical columns
categorical_columns = X.select_dtypes(include=['object']).columns
numerical_columns = X.select_dtypes(include=['int64', 'float64']).columns

# Instantiate the OneHotEncoder
#encoder = OneHotEncoder(sparse=False)
encoder = OneHotEncoder(sparse=False, drop='first')

# Fit and transform the categorical columns
encoded_categorical = encoder.fit_transform(X[categorical_columns])

# Convert the encoded array back to a DataFrame
encoded_df = pd.DataFrame(encoded_categorical, columns=encoder.get_feature_names_out(categorical_columns))

# Combine the encoded columns with the integer columns
final_df = pd.concat([encoded_df, X[numerical_columns].reset_index(drop=True)], axis=1)

# Display the final DataFrame
print(final_df)

       Month_Aug  Month_Dec  Month_Feb  Month_Jan  Month_Jul  Month_Jun  \
0            0.0        1.0        0.0        0.0        0.0        0.0   
1            0.0        0.0        0.0        1.0        0.0        0.0   
2            0.0        0.0        0.0        0.0        0.0        0.0   
3            0.0        0.0        0.0        0.0        0.0        1.0   
4            0.0        0.0        0.0        1.0        0.0        0.0   
...          ...        ...        ...        ...        ...        ...   
15415        0.0        0.0        0.0        0.0        0.0        0.0   
15416        0.0        0.0        0.0        0.0        0.0        0.0   
15417        0.0        0.0        0.0        0.0        0.0        0.0   
15418        0.0        1.0        0.0        0.0        0.0        0.0   
15419        0.0        1.0        0.0        0.0        0.0        0.0   

       Month_Mar  Month_May  Month_Nov  Month_Oct  ...  BasePolicy_Collision  \
0            0.0   

/opt/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


# Train Test Split

In [12]:
# check that the final_df does not have the label column

if 'FraudFound_P' in final_df.columns:
    print("FraudFound_P column is present in the DataFrame.")
else:
    print("FraudFound_P column is not present in the DataFrame.")


FraudFound_P column is not present in the DataFrame.


In [15]:
from sklearn.model_selection import train_test_split

# Split dataset into X_train and y_train
X_train, X_test, y_train, y_test = train_test_split(final_df, y, test_size=0.2, random_state=42)
#X_train, X_test, y_train, y_test = train_test_split(final_df.iloc[:,0:32], final_df.iloc[:,-1], test_size=0.2, random_state=52)


# XG-BOOST
- classify dataset. The score should be 80 or below!!!

In [18]:
# First, put this prompt: "conda install -c conda-forge py-xgboost" in anaconda to download xgboost package
# install xgboost in jupyter
!pip install xgboost


In [20]:
# import the classifier from the xgboost package
from xgboost import XGBClassifier

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics


# create model instance
# n_estimators: number of trees(estimators) the model uses --> the more used, the more accurate the model is
# max_depth: maximum depth of tree --> higher number makes model more complex, but too high can cause overfitting
# learning_rate: quantifies each tree's contribution to total prediction --> lower number takes longer, but can lead to better generalization
# objective: binary:logistic outputs probabilities. if classification is wanted, use binary:hinge
bst = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic', enable_categorical=True)

# fit model with the training data
bst.fit(X_train, y_train)

# make predictions for the test dataset
preds = bst.predict(X_test)

# print predictions
print("Predictions: ", preds)

# print model F1 score (how often the classifier is correct)
#print("Accuracy:",metrics.accuracy_score(y_test, preds))
f1 = metrics.f1_score(y_test, preds)
print("F1 Score:", f1)


Predictions:  [0 0 0 ... 0 0 0]
F1 Score: 0.0


In [22]:
#Import svm model
from sklearn import svm
from sklearn import metrics

#Create a svm Classifier
# kernel: options for kernel include linear, poly, rbf, sigmoid
    # linear: use this when data can be split by a linear function
    # poly (polynomial): use this when data can be split by a polynomial function
    # rbf (radial basis function): use this when there are clusters of one class inside another
    # sigmoid: use this when the split between classes is curved and irregular
clf = svm.SVC(kernel='linear')

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

# print predictions
print("Predictions: ", y_pred)

# print model Accuracy (how often the classifier is correct)
print("F1 Score:",metrics.f1_score(y_test, y_pred))



Predictions:  [0 0 0 ... 0 0 0]
F1 Score: 0.06837606837606837


In [24]:
#import knn model
from sklearn.neighbors import KNeighborsClassifier

#create a knn classifier
#n_neighbors: predicting the label of the data point by looking at the 3 closest data points and getting them to "vote"
#algorithm: we may need to look at this if it misbehaves
neigh = KNeighborsClassifier(n_neighbors=5)

#train the model using the training sets
neigh.fit(X_train, y_train)

#predict the response for the test dataset
y_pred = neigh.predict(X_test)

#print predictions
print("Predictions: " , y_pred)

#model accuracy (how often the classifier is correct)
f1 = metrics.f1_score(y_test, y_pred)
print("F1 Score: ", f1)

#print("Accuracy:",metrics.accuracy_score(y_test,y_pred))

Predictions:  [0 0 0 ... 0 0 0]
F1 Score:  0.00980392156862745


In [26]:
'''
Authors: Balachander S, Prahalad Srinivas C G, Yogesh Chandra Singh Samant, B Varshin Hariharan
'''
import numpy as np
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
from sklearn import datasets
import pandas as pd
import sklearn.metrics as skm

#import scikit learn packages
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

class FeatureClassifier:
  def __init__(self,reqAcc=0.01,classifier='DesicionTree',bias=[],control=None,n_jobs=None,random_state=None):
    self.featureClassifiers=[] #list of all the classifiers of all the selected features
    self.reqAcc=reqAcc #user specified cutoff value
    self.indexLs=[] # list of mapped index values to featureClassifiers
    self.flag=0
    self.bias=bias # list of biases for each and every label
    self.control=control #overfitting control for decision trees
    self.classifier=classifier #the classifier which is preferred by the user
    self.dic={'DecisionTree':0,'LinearRegression':1,'SVM':2,'LogisticRegression':3} #a dictionary which maps the classifier to its index
    self.n_jobs=n_jobs
    self.random_state=random_state
    self.num_lables = None

  def finIndex(self):
    #finds the index where the reqAcc condition fails and also created the indexLs[] for mapping
    for i in range(len(self.featureClassifiers)):
      if self.featureClassifiers[i][1] < self.reqAcc:
        return i
      self.indexLs.append(self.featureClassifiers[i][2])
    self.flag=1
    return i

  def fit(self,x,y):
    #applied the model to the dataset. The model is trained and saved for further prediction
    self.num_lables=len(set(y.flatten()))
    bestfeatures = SelectKBest(score_func=chi2,k=1)
    fit = bestfeatures.fit(x,y)

    for i in range(len(x[0])):
      clf=[DecisionTreeClassifier(max_depth=self.control,random_state=self.random_state),LinearRegression(n_jobs=self.n_jobs),SVC(gamma=self.control,random_state=self.random_state), LogisticRegression(penalty=self.control,random_state=self.random_state)][self.dic[self.classifier]]
      X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33,random_state=self.random_state)
      clf.fit(X_train[:,i:i+1],y_train)
      self.featureClassifiers.append((clf,fit.scores_[i],i))
    self.featureClassifiers.sort(key=lambda x:x[1],reverse=True)
    index=self.finIndex()
    if self.flag==0:
      self.featureClassifiers=self.featureClassifiers[:index]
    return

  def predict(self,x):
    #given a list of inputs, predicts the possible outputs
    if not self.bias:
      self.bias=np.zeros(self.num_lables)
    if len(self.bias)<self.num_lables:
      raise AttributeError('Please check the lenth of bias list')
    yPred=[]
    for i in range(len(x)):
      pred_arr=np.zeros(self.num_lables)
      for j in range(len(self.indexLs)):
        pred=np.round(self.featureClassifiers[j][0].predict([[x[i][self.indexLs[j]]]]))
        pred_arr[pred]+=self.featureClassifiers[j][1]+self.bias[pred[0]]
      yPred.append(np.argmax(pred_arr))
    return yPred
      
# Assuming FeatureClassifier is correctly implemented for DecisionTreeClassifier
clf1 = FeatureClassifier(0, classifier='DecisionTree', control=3, random_state=42)

# Fit model with the training data
clf1.fit(np.array(X_train), np.array(y_train).astype(int))

# Predict the response for the test dataset
y_pred = clf1.predict(np.array(X_test))

# Calculate and print F1 score
f1 = metrics.f1_score(np.array(y_test).astype(int), y_pred)
print("F1 Score:", f1)

# Print other relevant outputs for debugging
print(clf1.indexLs)

F1 Score: 0.0
[61, 59, 52, 8, 6, 60, 51, 15, 48, 45, 44, 4, 31, 50, 38, 13, 26, 0, 32, 34, 25, 1, 29, 19, 7, 14, 37, 27, 55, 35, 58, 40, 9, 12, 11, 53, 18, 17, 36, 21, 42, 22, 47, 23, 33, 3, 54, 30, 43, 57, 16, 41, 39, 56, 28, 46, 10, 2, 20, 24, 49, 5, 64, 81, 119, 115, 68, 106, 118, 109, 73, 83, 63, 84, 74, 114, 65, 93, 69, 78, 121, 102, 103, 91, 86, 104, 90, 70, 97, 105, 100, 88, 66, 82, 89, 62, 67, 87, 107, 108, 98, 85, 95, 122, 101, 116, 71, 111, 72, 92, 120, 110, 94, 113, 117, 96, 76, 77, 75, 99, 79, 123, 80, 112]
